In [83]:
import openvino as ov

import serial
import time

In [84]:
py_serial = serial.Serial(
    # Window
    port='COM4',
    # 보드 레이트 (통신 속도)
    baudrate=9600,
)

In [85]:

core = ov.Core()

model = core.read_model(model="model/person-detection-retail-0013.xml")
person_model = core.compile_model(model=model, device_name="CPU")

person_input_layer = person_model.input(0)
person_output_layer = person_model.output(0)
print("Input layer shape: ", person_input_layer.shape)
print("Output layer shape:", person_output_layer.shape)

Input layer shape:  [1,3,320,544]
Output layer shape: [1,1,200,7]


In [86]:
import cv2
import numpy as np

frame = cv2.imread("images/person.jpg")

resized_frame = cv2.resize(src=frame, dsize=(544, 320)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

In [87]:
person_output = person_model([input_frame])[person_output_layer]

In [90]:
def DrawBoundingBoxes(output, image, conf):

    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    // 사람의 수가 5명 이상이면 아두이도 LED를 초록색으로 변경
    if len(top_predictions) >= 5 :
        print("Green Light on")
        py_serial.write(b'b')   
    else: 
        print("Red Light on")
        py_serial.write(b'a')
    
    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    
    return canvas

In [91]:
canvas = DrawBoundingBoxes(person_output, frame, conf=0.5)

cv2.imshow("Person detection", canvas)

cv2.waitKey(0)
cv2.destroyAllWindows()

Green Light on


In [92]:
import cv2
cap = cv2.VideoCapture('./images/people2.mp4')
while cap.isOpened(): #동영상이 열려있는 동안 작동
    ret, frame = cap.read() #ret는 성공여부, frame은 받아온 이미지(프레임) 
    if not ret: # 받아오는 정보가 없다면
        print('영상 종료')
        break #종료
    
    resized_frame = cv2.resize(src=frame, dsize=(544, 320)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)

    person_output = person_model([input_frame])[person_output_layer]
    canvas = DrawBoundingBoxes(person_output, frame, conf=0.5)
    
    cv2.imshow('person detection',canvas) #영상 출력    
    if cv2.waitKey(60) == ord('q'): #waitKey() 안에 숫자를 조정하면 영상 재생 속도가 달라짐
        print('사용자 입력에 의해 종료')
        break
    
cap.release() #해제
cv2.destroyAllWindows() #종료

Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Light on
Green Ligh